# 10.3 Detecció d'objectes en una fase

## SSD

<table width=100%>
    <tr>
        <td style="border:1px solid black;background-color:#F0F0FF" width=100%>
            <h4>
                IDEA: Fusionar la proposició de finestres amb la detecció.
            </h4>
        </td>
    </tr>
</table>


Tot i els avenços en les diferents versions dels detectors de dues fases, el gran salt endavant va ser l'aparició dels detectors d'una sola fase que permetien localitzar i classificar a la vegada per un únic model. El **SSD** (de l'anglès *Single Shot MultiBox Detector*) va ser ser un dels primers models uni-fase amb un bon equilibri entre velocitat i precisió, i es va presentar al 2016.

El SSD utilitza una xarxa CNN organitzada en 3 parts:
1. Capes de convolució (xarxa VGG16) per extreure mapa de característiques de baix nivell de la imatge original.
2. Capes auxiliars per generar el mapa de característiques d'alt nivell.
3. Capes de predicció que localitzen i identifiquen els objectes en els mapes de característiques; capes que fan la predicció.

Les capes de la segona part, afegeixen diferents escalats al mapa de característiques de baix  nivell, similar al concepte d'imatge piramidal del Sliding Window, on cada capa afegeix un nivell més d'escalatge, anant de mapes més grans a un de menor dimensions. Cada sortida d'una capa d'escalatge es connecta, a més de amb la següent capa, també amb la capa de predicció, aconseguint així que el model pugui detectar objectes de diferents dimensions en un mateix model. La següent taula mostra les dimensions del model i l'arquitectura associada.

<table width=80%>
    <tr>
        <td style="border:1px solid black">
            <img src='Imatges/ssd-dimensions.png'>
        </td>
    </tr>
    <tr>
        <td style="border:1px solid black">
            <img src='Imatges/ssd.jpeg'>
        </td>
    </tr>
</table>


<table width=100%>
   <tr align=top>
        <td style="border:1px solid black;background-color:#F0F0FF">
            <h4>
                IDEA 2: No cal predir directament les bounding boxes sobre la imatge, podem treballar directament sobre el feature map.
            </h4>
        </td>
    </tr>
</table>

El SSD prediu les bounding boxes directament sobre el feature map, és a dir, sobre la sortida de la primera CNN que passem a la imatge sencera. A cada cel·la del feature map, el model aplicarà possibles *anchors* (àncores), que són bounding boxes de diferentes amplades i alçades. La següent taula mostra aquesta idea més gràficament.

<table width=70%>
    <tr>
        <td style="border:1px solid black">
            <img src='Imatges/ssd-8x8_feature_map.png'>
        </td>
        <td style="border:1px solid black">
            <img src='Imatges/ssd-anchors.png'>
        </td>
    </tr>
    <tr>
        <td style="border:1px solid black">
            Mapa de característiques 8x8. Per cada punt, podem predir una sèrie de bounding boxes 
        </td>
        <td style="border:1px solid black">
            Per a cada anchor (àncora), es fan una sèrie de prediccions de bounding boxes.
            Nota: Les àncores són només suposicions inicials de bounding boxes amb tamany limitat (per exemple quadrats i rectangles de diferents mides). Cada cel·la d'un feature map té associats diversos anchor (possibles bounding boxes).
        </td>
    </tr>
    <tr>
        <td colspan=2  style="border:1px solid black">
            <img src='Imatges/predictions_anchors.jpeg'>
        </td>
    </tr>
    <tr>
        <td colspan=2  style="border:1px solid black">
            Exemple de diferents bounding boxes possibles a partir d'un mateix anchor. Normalment les bounding box rectangulars en posició vertical acostumen a identificar persones, mentre que les rectangulars apaïsades detecten més cotxes, bicicletes, gossos. Això també es té en compte a l'hora de classificar l'objecte trobat.
        </td>
    </tr>
</table>


* Propietats
 - Més ràpids que Faster-RCNN, però a vegades menys precisos que aquests.
 - A diferència de Faster-RCNN, el SSD prediu N classes a la vegada
 - Tenen problemes per detectar objectes molt petits i propers entre ells, la raó principal és l'escalatge de les capes convolucionals intermèdies de forma piramidal (cada cop una capa més petita). La majoria de vegades una imatge original a major resolució acostuma a solucionar el problema (així les capes finals de la piràmide encara poden veure els objectes petits amb suficient resolució).



## YOLO

<table width=100%>
    <tr>
        <td style="border:1px solid black;background-color:#F0F0FF" width=100%>
            <h4>
                IDEA: Buscar només bounding boxes amb alta probabilitat de contenir un objecte.
            </h4>
        </td>
    </tr>
</table>


YOLO (de l'anglès *You Only Look Once*) és un dels últims i més populars algoritmes de detecció d'objectes; va ser creat originalment per detectar objectes en temps real.

Passos principals del model:
1. YOLO divideix la imatge original en una graella de $\begin{equation} SxS \end{equation}$ quadrats (imatge de l'esquerra). 
2. Per a cada quadrícula, agafa $\begin{equation}N\end{equation}$ possibles capses delimitadores (bounding boxes).
3. Per a cada bounding box la xarxa calcula un nivell de certesa o probabilitat de classe (imatge del centre). Es calculen $\begin{equation}SxSxN\end{equation}$ diferents capses, la majoria d'elles amb nivells molts baixos de certesa.
4. S'eliminen les capses amb valors per sota d'un cert llindar (nivell de certesa baix)
5. A les capses restants se'ls aplica un pas de "non-max suppression" que serveix per eliminar possibles objectes detectats per duplicat i quedar-se amb el més exacte d'ells (imatge de la dreta)


<table>
    <tr>
        <td style="border:1px solid black">
            <img src="Imatges/yolo.png">
            <br>YOLO<br>Font: <a href="Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi. You Only Look Once: Unified, Real-Time Object Detection">Joseph Redmon et al.</a>
        </td>
    </tr>
</table>
            
#### Non-maximum suppression
Quan es busquen i es troben RoI o capses que realment contenen objectes, és molt comú que hi hagi sobreposició entre diversos candidats; el non-maximum supression (NMS) és una tècnica que busca el millor candidat i elimina la resta, i assegura que en cada RoI hi ha un únic objecte detectat.

<table width=80%>
    <tr><td><img src='Imatges/nms.png'></td></tr>
</table>

YOLO és molt més ràpid que altres detectors d'objectes en temps real; pot arribar a processar 45 frames per segon. Tot i que sacrifica una mica la precisió a canvi de guanyar en velocitat; la seva principal limitació és que no processa bé objectes petits dins la imatge (com per exemple una bandada d'ocells)

Fins avui ja s'han implementat 3 versions del YOLO, cadascuna d'elles millorant en alguna mesura l'anterior versió: YOLO, YOLOv2/YOLO9000 i, YOLOv3.

## RetinaNet
Finalment, us presentem una de les xarxes més noves, la RetinaNet, que no va introduir grans canvis en la metodologia ni en l'arquitectura dels models (de fet era una SSD), però va canviar la funció de pèrdua (loss function) aconseguint una gran millora en la precisió del model. 

Un dels problemes dels detectors d'objectes en temps real és la detecció de falsos positius on no hi ha cap objecte d'interès a detectar, és a dir, que principalment són bounding boxes amb fons de imatge (el background). Es creu que la confusió del model entre objectes a detectar i el background fa que els algoritmes de detecció en una fase siguin menys precisos que els models de dues fases, tot i que molt més ràpids. Té la seva lògica, en els models de 1 fase hi ha una primera fase on es troben els bounding boxes amb objectes, i una segona on es classifiquen aquests objectes. Normalment, els bounding boxes amb objectes tenen molt poc background, ja que la idea és que la capsa delimitadora sigui ocupada en la seva gran majoria per l'objecte detectat. Ara bé, en el cas dels models d'una fase, es processa tota la imatge a la vegada a través de una CNN, és a dir, que la majoria de contingut a processar és background i, molt important, les CNN tenen tendència a tenir desviació cap a les classes que apareixen més sovint.

El RetinaNet va millorar aquest problema només introduint una nova funció de pèrdua (loss function) anomenada **focal loss** que penalitza els exemples classificats fàcilment (i, per tant, el background en la majoria dels casos). La següent gràfica mostra una comparativa de la precisió aconseguida en el RetinaNet respecte a xarxes anteriors, fa servir el mAP com a mètrica d'avaluació per comparar la precisió dels resultats, mètrica que us expliquem en la següent secció d'aquest capítol.



<table width=75%>
    <tr>
        <td style="border:1px solid black">
            <img src="Imatges/comparativa2.png">
        </td>
    </tr>
    <tr>
        <td>
            L'eix de les X mostra el temps de validació o inferència dels resultats, mentre que l'eix de les Y mostra la precisió basada en mAP. (Font: <a href="https://medium.com/@jonathan_hui/object-detection-speed-and-accuracy-comparison-faster-r-cnn-r-fcn-ssd-and-yolo-5425656ae359">Medium</a>)
        </td>
    </tr>
</table>




## Mètriques d'avaluació per detecció d'objectes: mAP

Una avaluació mètrica molt utilitzada en models de detecció d'objectes és la coneguda com **mAP** (de l'anglès *mean Average Precision*) que es basa en les àrees coincidents les prediccions respecte les reals, també conegudes com ground-truth bounding boxes.

Nota: les ground-truth bounding boxes són les mínimes capses o finestres que rodegen un objecte i que s'han creat i etiquetat manualment amb l'objectiu d'entrenar el sistema.

<table width=400>
    <tr>
        <td style="border:1px solid black"><img src="Imatges/iou_stop_sign.jpg"></td>
    </tr>
    <tr>
        <td style="border:1px solid black" align=center>La capsa predicció en color vermell i la ground-truth bounding box en color verd. <br>
        Font: <a href="https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/">PyImageSearch</a>
        </td>
    </tr>    
</table>


Característiques del mAP:
* Dóna un  número entre 0 i 100; a nombres més elevats, més nivell de precisió.
* Combina totes les deteccions d'objectes de totes les imatges de test per dibuixar una corba de *precision-recall* (corba PR) per cada classe; l'àrea de sota la corba és la mitjana de precisió, coneguda com *AP* per l'anglès *Average Precision*.
* Tenint en compte que els objectes a trobar es troben en diferents classes, primer calculem AP per separat per a cada classe, i després fem una mitjana sobre les classes.
* Una detecció d'objecte és un positiu (verdader positiu) si té una "intersecció sobre unió" (**IoU** de l'anglès *Intersection over Union*) major que algun llindar específic (normalment 0,5).

El IoU és una mesura de precisió per la detecció d'objectes basada en la quantitat d'àrea de la finestra predicció que coincideix amb la finestra d'entrenament:


<table width=80%>
    <tr>
        <td style="border:1px solid black" width=45%><img src="Imatges/iou_equation.png"></td>       
        <td style="border:1px solid black"><img src="Imatges/iou_examples.png"></td>       
    </tr>
    <tr>
        <td style="border:1px solid black" align=center>El càlcul de l'IoU és la divisió entre l'àrea d'intersecció i l'àrea d'unió de les capses base i predites.</td>
        <td style="border:1px solid black" align=center>Exemples de valors de IoU.</td>
    </tr>  
    <tr>
        <td colspan=2 align=center style="border:1px solid black">
            Font: <a href="https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/">PyImageSearch</a>
        </td>
    </tr>
</table>

# Referències

- **SSD**: 
 - Bona explicació i interessant comparativa de SSD amb altres algoritmes: [https://towardsdatascience.com/review-ssd-single-shot-detector-object-detection-851a94607d11](https://towardsdatascience.com/review-ssd-single-shot-detector-object-detection-851a94607d11)
 - Article acadèmic: Liu, Wei et al. “*SSD: Single Shot MultiBox Detector.*” Lecture Notes in Computer Science (2016): 21–37. Crossref. Web: [https://arxiv.org/abs/1512.02325](https://arxiv.org/abs/1512.02325)

- **YOLO**: 
 - 1a versió, l'original: Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi. *You Only Look Once: Unified, Real-Time Object Detection*" (2016): [https://arxiv.org/pdf/1506.02640.pdf](https://arxiv.org/pdf/1506.02640.pdf)
 - 2a versió: "*YOLO9000: Better, Faster, Stronger*" (2017): [https://arxiv.org/pdf/1612.08242.pdf](https://arxiv.org/pdf/1612.08242.pdf)
 - 3a versió: "*YOLOv3: An Incremental Improvement*" (2018) [https://pjreddie.com/media/files/papers/YOLOv3.pdf](https://pjreddie.com/media/files/papers/YOLOv3.pdf)


- Explicació i implementació de l'**IoU** en Python: "*Intersection over Union (IoU) for object detection*" [https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/](https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/)

- **RetinaNET** (Focal Loss): 
 - Explicació intuïtiva: [https://towardsdatascience.com/neural-networks-intuitions-3-focal-loss-for-dense-object-detection-paper-explanation-61bc0205114e](https://towardsdatascience.com/neural-networks-intuitions-3-focal-loss-for-dense-object-detection-paper-explanation-61bc0205114e)
 - Article acadèmic: Tsung-Yi Lin et al. "*Focal Loss for Dense Object Detection*" (2017): [https://arxiv.org/abs/1708.02002](https://arxiv.org/abs/1708.02002)